In [24]:
import numpy as np
import math

a = 1
b = 5
c = 1
p = 1
Ncount = 49

matrixForSys = []

def putZerosInArray (arr,count):
    for _ in range(0,count):
        arr.append (0)
        
def buildNewSystemOfEquations (N):
    i = 0
    
    f = i + 1
    tmpRow = []
    tmpRow.append(b)
    tmpRow.append(c) 
    putZerosInArray(tmpRow,(N+1)-2)
    tmpRow.append(f) 
    matrixForSys.insert(0,tmpRow)
    

    for k in range(1,N):
        f = k + 1
        tmpRow = []
        
        if k>1:
            putZerosInArray(tmpRow,k-1)
            
        tmpRow.append(a)
        tmpRow.append(b)
        tmpRow.append(c)     
        putZerosInArray(tmpRow,(N+1)-k-2)
        tmpRow.append(f) 
        matrixForSys.insert(k,tmpRow)

    tmpRow = None
    tmpRow = []
    for i in range (1,N+2):
        tmpRow.append(p)
    tmpRow.append(N+1)
    matrixForSys.append(tmpRow)


buildNewSystemOfEquations(Ncount)

def KOSTblLcopy (matrix):
    copyed = []
    for i in range(0,len(matrix)):
        tmp = []
        for k in range(0,len(matrix[i])):
                tmp.append(matrix[i][k])
        copyed.append(tmp)
    return copyed 

def KOSTblLcopyArray (arr):
    tmp = []
    for i in range(len(arr)):
            tmp.append(arr[i])
    return tmp 

def splitMatrixToAandB (matrix):
    A = []
    B = []
    for i in range(0,len(matrix)):
        tmp = []
        for k in range(0,len(matrix[i])):
            if k<(len(matrix[i])-1):
                tmp.append(matrix[i][k])
            else:
                B.append(matrix[i][k])
        A.append(tmp)
        
    return A,B


def getChisloObyslovlennosti (matrix):
    A,_ = splitMatrixToAandB(matrixForSys)

    det = np.linalg.det(A)
    obratnaia = np.linalg.inv(A)
    detO = np.linalg.det(obratnaia)
    return (det*detO)
    
def getSobstvennoeZnachenie (matrix):
    e_vals, _ = np.linalg.eig(matrix)
    print("--------------------------------------------")
    print("Lambda min = " + str(min(e_vals)) + "     Lambda max = "+ str(max(e_vals)))
    print("--------------------------------------------")


print("Chislo obyslovlennosti: " +str(getChisloObyslovlennosti (matrixForSys))   ) 
    
def GaussSeidel(A, B, eps):
    N = len(A)
    X = []
    for i in range(N):
        X.append(.0)
    cycle = True
    while cycle:
        Xnew = KOSTblLcopyArray(X)

        for i in range(N):
            summ1, summ2 = 0,0 
            for j in range(i):
                summ1+=A[i][j] * Xnew[j]
            for j in range(i + 1, N):
                summ2+=A[i][j] * X[j]
                
            Xnew[i] = (B[i] - summ1 - summ2) / A[i][i]

        norm = 0
        for k in range(N):
            norm+= (Xnew[i] - X[i]) **2
            if math.sqrt(norm) <=eps:
                print("------------------------------------------")
                print("cret ostan","|",Xnew[i],"-",X[i],"|<=",eps)
                cycle = False
                break
        X = Xnew

    return X

def swapRows (A, B, first, second):
    A[first], A[second], B[first], B[second] = A[second], A[first], B[second], B[first] 

    
    
def divCoeff (A, B, row, divider):
    num = 0
    for a in A[row]:
        A[row][num] = a / divider 
        num+=1
    B[row] /= divider

def decrRow (A, B, row, source_row, weight):

    for i in range(0,len(A[0])):
        A[row][i] = (A[row][i] + A[source_row][i] * weight)
    B[row] += B[source_row] * weight


def Gauss (A, B):

    column = 0
    while(column < len(B)):
        current_row = None
        for i in range(column, len(A)):
            if current_row is None or abs(A[i][column]) > abs(A[current_row][column]):
                current_row = i
        if current_row != column:
            swapRows (A, B, current_row, column)
        divCoeff (A, B, column, A[column][column])
        for i in range(column+1, len(A)):
            decrRow(A, B, i, column, -A[i][column])
        column += 1
        
    X = KOSTblLcopyArray(B)
    
    for i in range(len(B)-1, -1, -1):
            X[i] = B[i] - sum(x*a for x,a in zip(X[(i+1):], A[i][(i+1):]))
        
    otvet = ""
    for i in range(len(X)):
        otvet = otvet + "X"+str(i+1) +" = "+str(X[i])+", " 
    print("METHOD GAUSS: otvet: "+otvet)
    print("--------------------------------------------")
    return X
    

def Jacobi ( A,  B, eps):
    N = len(A)
    
    X =[]
    for i in range(0,len(A)):
        X.append(B[i]/A[i][i])    

    tmpX = []
    
    cycle = True 
    while cycle:
        for i  in range(0,N):
            tmpX.insert(i,B[i])
            for g in range(0,N):
                if (i != g):
                    tmpX[i] -= A[i][g] * X[g];

            tmpX[i] /= A[i][i]

        norm = abs(X[0] - tmpX[0])
        for j in range(0,N):
            if (abs(X[j] - tmpX[j]) > norm):
                norm = abs(X[j] - tmpX[j])
            X[j] = tmpX[j]

        if not (norm > eps):
            otvet = ""
            for i in range(len(X)):
                otvet = otvet + "X"+str(i+1) +" = "+str(X[i])+", " 
            print("METHOD JACOBI: otvet: "+otvet)
            cycle = False 
    return X

def getNeviazka (data,name):
    finalData = []
    for i in range(len(matrixForSys)):
        summ = 0
        finalData.insert(i,[])
        for j in range(len(matrixForSys[0])-1):
            summ+= matrixForSys[i][j]*data[j]
        finalData[i].insert(0,summ)
        finalData[i].insert(1,matrixForSys[i][len(matrixForSys[0])-1])
        finalData[i].insert(2,finalData[i][1]-finalData[i][0])
    print("--------------------------------------------")
    print( "Method " +name)        
    for i in range(len(matrixForSys)):
        print("(  "+str(finalData[i][0])+ " = " +str(finalData[i][1]) +"  ) neviazka: "+str(finalData[i][2]))
    print("--------------------------------------------")
    return True


A,B = splitMatrixToAandB (matrixForSys)
getSobstvennoeZnachenie(A)

resG = Gauss (A, B)

resJ = Jacobi ( A,  B,  0.0001)

##################################
resGS = GaussSeidel(A, B, 0.0001)
otv = ""
for i in range(len(resG)):
    otv = otv + "X"+str(i+1) +" = "+str(resGS[i])+", " 
print("--------------------------------------------")
print("METHOD GAUSS-SEIDEL: otvet: "+otv)

getNeviazka(resG, "Gauss")
getNeviazka(resJ, "Jacobi")
getNeviazka(resGS, "Gauss-Seidel")

Chislo obyslovlennosti: 1.0000000000000568
--------------------------------------------
Lambda min = 0.7979319694208473     Lambda max = 7.023592375788785
--------------------------------------------
METHOD GAUSS: otvet: X1 = 0.14285714285714288, X2 = 0.2857142857142857, X3 = 0.42857142857142855, X4 = 0.5714285714285714, X5 = 0.7142857142857143, X6 = 0.857142857142857, X7 = 1.0000000000000002, X8 = 1.1428571428571428, X9 = 1.285714285714286, X10 = 1.4285714285714284, X11 = 1.5714285714285714, X12 = 1.714285714285714, X13 = 1.857142857142857, X14 = 2.0, X15 = 2.1428571428571423, X16 = 2.2857142857142856, X17 = 2.428571428571429, X18 = 2.571428571428571, X19 = 2.7142857142857144, X20 = 2.8571428571428568, X21 = 2.9999999999999996, X22 = 3.142857142857143, X23 = 3.285714285714285, X24 = 3.4285714285714284, X25 = 3.571428571428573, X26 = 3.7142857142857064, X27 = 3.857142857142893, X28 = 3.9999999999998286, X29 = 4.142857142857962, X30 = 4.285714285710356, X31 = 4.428571428590255, X32 = 4.

True